# Array, Variable, and Operators

@[Xiaoyu Chen](mailto:c-xy17@tsinghua.org.cn)
@[Chaoming Wang](mailto:chao.brain@qq.com)

In this section, we will briefly introduce two basic data structures: `Array` and `Variable`. They form the foundation for mathematical operations of brain dynamics programming (BDP) in BrainPy.

In [1]:
import brainpy as bp
import brainpy.math as bm

# bm.set_platform('cpu')

In [2]:
bp.__version__

'2.3.0'

## Array

`brainpy.math.Array` is a wrapper of `jax.numpy.ndarray`. It aims to provide a consistent experience with `numpy.ndarray`.

Generally speaking, `brainpy.math.Array` is the same as `numpy.ndarray`. It supports almost the same attributes and operations as those for `numpy.ndarray`.

Below we only show basic examples. More details please refer the NumPy's official documentation.

### Definition and Attributes

In [2]:
t1 = bm.array([[[0, 1, 2, 3], [1, 2, 3, 4], [4, 5, 6, 7]], 
               [[0, 0, 0, 0], [-1, 1, -1, 1], [2, -2, 2, -2]]])
t1

Array([[[ 0,  1,  2,  3],
        [ 1,  2,  3,  4],
        [ 4,  5,  6,  7]],

       [[ 0,  0,  0,  0],
        [-1,  1, -1,  1],
        [ 2, -2,  2, -2]]], dtype=int32)

Here we create a 3-dimensional array with the shape of (2, 3, 4) and the type of `int32`. Arrays created by ``brainpy.math`` will be stored in ``Array``, for their future operations will be accelerated by just-in-time (JIT) compilation.

A array has several important attributes:

- **.ndim**: the number of axes (dimensions) of the array.

- **.shape**: the dimensions of the array. This is a tuple of integers indicating the size of the array in each dimension. For a matrix with n rows and m columns, the shape will be `(n,m)`. The length of the shape tuple is therefore the number of axes, `ndim`.

- **.size**: the total number of elements of the array. This is equal to the product of the elements of shape.

- **.dtype**: an object describing the type of the elements in the array. One can create or specify dtypes using standard Python types.

In [3]:
print('t1.ndim: {}'.format(t1.ndim))
print('t1.shape: {}'.format(t1.shape))
print('t1.size: {}'.format(t1.size))
print('t1.dtype: {}'.format(t1.dtype))

t1.ndim: 3
t1.shape: (2, 3, 4)
t1.size: 24
t1.dtype: int32


Below we will give a few examples of array operations that are commonly used in brain dynamics programming.

### Creating a array

In [4]:
t2 = bm.arange(4)
# t2: Array([0, 1, 2, 3], dtype=int32)

t3 = bm.ones((2, 4)) * 1.5
# t3: Array([[1.5, 1.5, 1.5, 1.5],
#               [1.5, 1.5, 1.5, 1.5]], dtype=float32)

### Array operations

In [5]:
# indexing and slicing
t3[1]

DeviceArray([1.5, 1.5, 1.5, 1.5], dtype=float32)

In [6]:
t3[:, 2:]

DeviceArray([[1.5, 1.5],
             [1.5, 1.5]], dtype=float32)

In [7]:
# algebraic operations
t2 + t3[0]

Array([1.5, 2.5, 3.5, 4.5], dtype=float32)

In [8]:
t3[0] / t1[0, 1]

DeviceArray([1.5  , 0.75 , 0.5  , 0.375], dtype=float32)

In [9]:
# broadcasting
t2 + 3

Array([3, 4, 5, 6], dtype=int32)

In [10]:
t2 + t3

Array([[1.5, 2.5, 3.5, 4.5],
       [1.5, 2.5, 3.5, 4.5]], dtype=float32)

In [11]:
# some functions
bm.dot(t2, t3.T)

Array([9., 9.], dtype=float32)

In [12]:
bm.max(t1, axis=2)

Array([[3, 4, 7],
       [0, 1, 2]], dtype=int32)

In [13]:
t3.flatten()
# Array([1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5], dtype=float32)

Array([1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5, 1.5], dtype=float32)

In BrainPy, arrays can be used to store some parameters related to dynamical models. For example, if we define a group of Integrate-and-Fire (LIF) neurons and wish to assign each neuron with a different time constant $\tau$, then we can generate an array containing an array of time constants.

In [14]:
n = 6  # assume there are 6 LIF neurons
tau = bm.random.randn(n)*2. + 20.
tau

Array([17.865358, 21.120678, 21.62801 , 20.729681, 22.94931 ,
       18.765892], dtype=float32)

Through the code above, a group of time constants is generated from a normal distribution, with a mean of 20 and a variance of 2.

## Variable

We have talked about the definition, operations, and application of arrays in BrainPy. There are some situations, however, where arrays are not applicable. Due to JIT compilation, static arrays will be compiled as the static values. If you want to change the value of an array, you should name it as a ``brainpy.math.Variable``. Variable tells the JIT compiler that this array should not a static value.

### ``brainpy.math.Variable``

``brainpy.math.Variable`` is a pointer referring to an array. The array is stored as its value. The data in a `Variable` can be changed during JIT compilation. **If an array is labeled as a Variable, it means that it is a dynamical variable that changes during the function call.**

To create or change a array into a variable, users just need to wrap the array into ``brainpy.math.Variable``:

In [15]:
v = bm.Variable(t2)
v

Variable([0, 1, 2, 3], dtype=int32)

Note that the array is contained in a "Variable" instead of a "Array".

```{note}
Arrays that are not marked as Variables will be JIT compiled as static data. This will cause errors and wrong results.
```

Users can access the value in the Variable through its attribute `.value`:

In [16]:
v.value

DeviceArray([0, 1, 2, 3], dtype=int32)

Since the data inside a Variable is a array, common operations on arrays can be directly grafted to Variables.

### In-place updating

Though the operations are the same, there are some requirements for updating a Variable. If we directly change a Variable, The returning data will become a array but not a Variable.

In [17]:
v2 = v + 2
v2

DeviceArray([2, 3, 4, 5], dtype=int32)

To update the Variable, users are required to use in-place updating, which only modifies the value inside the Variable but does not change the reference pointing to the Variable. In-place updating operations include:

**1\. Indexing and slicing**

  - Indexing: ``v[i] = a``
  - Slicing: ``v[i:j] = b``
  - Slicing the specific values: ``v[[1, 3]] = c``
  - Slicing all values, ``v[:] = d``, ``v[...] = e``

for more details, please refer to [Array Objects Indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html).

In [18]:
v[0] = 10
v[1:3] = 9
v

Variable([10,  9,  9,  3], dtype=int32)

**2\. Augmented assignment**

- ``+=`` (add)
- ``-=`` (subtract)
- ``/=`` (divide)
- ``*=`` (multiply)
- ``//=`` (floor divide)
- ``%=`` (modulo)
- ``**=`` (power)
- ``&=`` (and)
- ``|=`` (or)
- ``^=`` (xor)
- ``<<=`` (left shift)
- ``>>=`` (right shift)

In [19]:
v -= 3
v <<= 1
v

Variable([14, 12, 12,  0], dtype=int32)

**3\. ``.value`` assignment**

In [20]:
v.value = bm.arange(4)
v

Variable([0, 1, 2, 3], dtype=int32)

`` .value`` assignment directly accesses the data stored in the Array. When using `.value`, the new data should be of the same type and shape as the original ones.

In [21]:
try:
    v.value = bm.array([1., 1., 1., 0.])
except Exception as e:
    print(type(e), e)

<class 'brainpy.errors.MathError'> The dtype of the original data is int32, while we got float32.


**4\. ``.update()`` method**

This method will also check if the new data is of the same type and shape as the original ones.

In [22]:
v.update(bm.array([3, 4, 5, 6]))
v

Variable([3, 4, 5, 6], dtype=int32)

## Operators

### Dense matrix-based operators

`Array` and `Variable` support all dense matrix-based operators as in NumPy. For example:

1. Mathematical functions

In [23]:
a = bm.array([20, 30, 40, 50])
b = bm.arange(4)

In [24]:
a - b

Array([20, 29, 38, 47], dtype=int32)

In [26]:
10 * bm.sin(a)

Array([ 9.129453 , -9.880316 ,  7.4511313, -2.6237485], dtype=float32)

In [27]:
b**2

Array([0, 1, 4, 9], dtype=int32)

In [28]:
a < 35

Array([ True,  True, False, False], dtype=bool)

2. Linear algebra functions

In [29]:
A = bm.array([[1, 1],
              [0, 1]])
B = bm.array([[2, 0],
              [3, 4]])

In [30]:
A * B     # elementwise product

Array([[2, 0],
       [0, 4]], dtype=int32)

In [31]:
A @ B     # matrix product

Array([[5, 4],
       [3, 4]], dtype=int32)

In [32]:
A.dot(B)  # another matrix product

Array([[5, 4],
       [3, 4]], dtype=int32)

3. Discrete Fourier Transform

In [33]:
bm.fft.fft(a)

Array([140. +0.j, -20.+20.j, -20. +0.j, -20.-20.j], dtype=complex64)

4. Random sampling

In [35]:
bm.random.rand(2, 5)

Array([[0.70130324, 0.04279745, 0.9619278 , 0.97293615, 0.4039507 ],
       [0.2654785 , 0.21006942, 0.68680966, 0.40777338, 0.6556951 ]],            dtype=float32)

### Sparse and event-based operators

`Array` and `Variable` also support sparse and event-based operations provided in [brainpylib](https://brainpylib.readthedocs.io/en/latest/).

These operators are designed to meet the special needs (specifically, the sparse computation and event-driven computation) in brain dynamics modeling.

More details we recommend users to read the documentation of `brainpylib`.

## Further reading

For more details about the feature of `Array` and its `operations`, we highly recommend to read:

- [NumPy quickstart](https://numpy.org/devdocs/user/quickstart.html)
- [NumPy: the absolute basics for beginners](https://numpy.org/devdocs/user/absolute_beginners.html)

All these features and operations can be available in `brainpy.math`.